In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Khai báo thư viện

In [ ]:
import lzma
import os
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import re

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Giải nén dữ liêụ

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!unzip -o /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip
!unzip -o /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.tsv.zip

# Phân tích dữ liệu

In [ ]:
df_train = pd.read_csv('train.tsv', sep='\t')
df_test = pd.read_csv('test_stg2.tsv', sep='\t')

df_train.drop(df_train[df_train['price'] == 0].index, inplace=True)

In [ ]:
print("Train shape:", df_train.shape)
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
print("Test shape:", df_test.shape)
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
print(df_train['name'].value_counts())

In [ ]:
print(df_train['item_condition_id'].value_counts())

In [ ]:
print(df_train['category_name'].value_counts())
print("Category_name missing:", df_train['category_name'].isnull().sum())

In [ ]:
print(df_train['brand_name'].value_counts())
print("Brand_name missing:", df_train['brand_name'].isnull().sum())

In [ ]:
df_train['price'].describe()


In [ ]:
print(df_train['shipping'].value_counts())


In [ ]:
print(df_train['item_description'].value_counts())
print("item_description missing:", df_train['item_description'].isnull().sum())

# Preprocess

In [ ]:
def clean_df(df):
    # Xử lỹ những dữ liệu bị thiếu
    df['brand_name']=df['brand_name'].fillna('Unknown')
    df['category_name']=df['category_name'].fillna('Unknown/Unknown/Unknown')
    df['item_description']=df['item_description'].fillna('Unknown')
    df['item_condition_id']=df['item_condition_id'].fillna(3)
    
     # Cắt chuỗi trong trường 'category_name' và thêm các trường 'category1', 'category2', 'category3','category12'
    df[['category1', 'category2', 'category3']] = df['category_name'].str.split('/', 2, expand=True)
    df['category12'] = df['category1'].astype(str)+'_'+df['category2'].astype(str)

    # Thêm trường dữ liệu là độ dài của 'name' và độ dài của 'item_description'
    df['name_length'] = df['name'].str.len()
    df['item_description_length'] = df['item_description'].str.len()

    # Tìm chuỗi 'brand new' và 'tag' trong trường 'item_description' và thêm trường 'word_brand_new', 'word_tag'
    df['word_brand_new'] = df['item_description'].str.lower().str.contains('brand new')
    df['word_brand_new'] = df['word_brand_new'].astype(int)
    df['word_tag'] = df['item_description'].str.lower().str.contains('tag').astype(int)
    df['word_tag'] = df['word_tag'].astype(int)

    return df
    
df_train = clean_df(df_train)
df_test = clean_df(df_test)

In [ ]:
# Nhóm tên và giá, sau đó trích xuất các cặp để truy xuất nhanh hơn
def create_price_d(grouped, operation_name, orig_colname):
    price_d = {}
    for index, row in grouped.iterrows():
        price_d[row[orig_colname]] = row['price']
    return {orig_colname + "_" + operation_name: price_d}


# Tạo ra các bản tra cứu dựa trên 'brand_name','category1','category12','category_name' và sử dụng nó như công cụ định giá
price_d = {}    
for col in ['category1', 'category12', 'category_name', 'brand_name']:
    price_d.update(create_price_d(df_train.groupby(col, as_index=False).min(), 'min', col))
    price_d.update(create_price_d(df_train.groupby(col, as_index=False).median(), 'median', col))
    price_d.update(create_price_d(df_train.groupby(col, as_index=False).mean(), 'mean', col))
    price_d.update(create_price_d(df_train.groupby(col, as_index=False).max(), 'max', col))

In [ ]:
# Thêm các tùy chỉnh trên và df_train và df_test
def price_df(df, pricedicts):
    for col in ['brand_name', 'category1', 'category12', 'category_name']:
        for oper in ['_min', '_median', '_mean', '_max']:
            df[col + oper] = df[col].map(price_d[col + oper])
            df[col + oper] = df[col + oper].fillna(price_d[col + oper].get('Unknown', 5))

price_df(df_train, price_df)
price_df(df_test, price_df)

# Mô hình

In [ ]:
# Tạo dữ liệu cho mô hình
xs = ['item_condition_id', 'shipping', 'name_length', 'item_description_length',
      'brand_name_min', 'brand_name_max', 'brand_name_median', 'brand_name_mean', 
      'category1_min', 'category1_max', 'category1_median', 'category1_mean',
      'category12_min', 'category12_max', 'category12_median', 'category12_mean',
      'category_name_min', 'category_name_max', 'category_name_median', 'category_name_mean','word_brand_new', 'word_tag']

df_train_xs = df_train[xs]
df_train_y = df_train[['price']]

df_test_xs  = df_test[xs]

In [ ]:
# LinearRegssion
reg = linear_model.LinearRegression()

# Train mô hình sử dụng tập train
reg.fit(df_train_xs, df_train_y)

# Tạo ra dự đoán dựa trên tập test
df_test['price'] = reg.predict(df_test_xs)

# Với những dự đoán giá < 0, sửa chúng thành 0
df_test.loc[df_test.price < 0, 'price'] = 0

# Các hệ số của mô hình
print('Coefficients: \n', reg.coef_)

# Submit

In [ ]:
submissiondf = df_test[['test_id', 'price']]
submissiondf.to_csv('submission.csv', index=False)

submissiondf.head(10)